In [1]:
import pandas as pd
import numpy as np
import re


In [2]:
df = pd.read_csv("65075.csv")


In [3]:
def get_patern_of_bullet(String):
    regx = [
        ('\d{5}-\d+$', 70),
        ('[1-9][0-9]*(\.[1-9][0-9]*)*\)$', 20),
        ('\(\d*(\.?\d*)*\)$', 50),
        ('[1-9][0-9]*(\.[1-9][0-9]*)+$', 2),
        ('[1-9][0-9]*\.$', 1),
        ('[1-9][0-9]*$', 30)
    ]

    for r, l in regx:
        if re.match(r, String):
            if l in [2, 20, 50]:
                l = String.count('.') + l
            return r, l
    return '', 0


In [12]:
job = ''
bulletFlag = False
stack = []
entries = []
entry = []
for filename in df.filename.unique():
    # print(filename)
    page = df[df['filename'] == filename][1:]
    lines = [[] for i in range(page.line_num.max()+1)]
    for i in page.index:
        lines[page.loc[i].line_num].append(i)
    for i in lines[::-1]:
        lineText = df.loc[i].text.values
        bullet = get_patern_of_bullet(df.loc[i[0]].text.split(' ')[0])

        if lineText[0].startswith('งาน:'):
            if (lineText[-1] == 'บาท'):
                lineText = lineText[:-2]
            job = ' '.join(lineText)
        elif bullet[1]:
            bulletFlag = True

        if bulletFlag:
            entry += i
            if lineText[-1] == 'บาท':
                # print(*df.loc[entry].text.values)
                bulletFlag = False
                entry_bullet = get_patern_of_bullet(
                    df.loc[entry[0]].text.split(' ')[0])
                # print(entry_bullet[1], df.loc[entry[0]].text.split(' ')[0])
                e = (entry, entry_bullet[1])
                if len(stack) == 0:
                    stack.append(e)
                elif e[1] > stack[-1][1]:
                    stack.append(e)
                elif e[1] == stack[-1][1]:
                    entries.append(
                        {'job': job, 'nodes': [s[0].copy() for s in stack]})
                    stack.pop()
                    stack.append(e)
                else:
                    entries.append(
                        {'job': job, 'nodes': [s[0].copy() for s in stack]})
                    while len(stack) > 0 and e[1] <= stack[-1][1]:
                        stack.pop()
                    stack.append(e)
                entry = []

entries.append({'job': job, 'nodes': [s[0].copy() for s in stack]})


In [13]:
def toRow(entry):
    row = {}
    row['job'] = entry['job']
    levels = ['L1', 'L2', 'L3', 'L4', 'L5']
    for i, node in enumerate(entry['nodes']):
        row[levels[i]] = ' '.join(df.loc[node].text.values)
    row['amount'] = df.loc[entry['nodes'][-1][-2]].text
    return row


In [14]:
pd.DataFrame([toRow(e) for e in entries])


,job,L1,L2,L3,amount
0,งาน: อํานวยการและบริหารสํานักงานเขต,"1. งบบุคลากร 4,028,000 บาท","1.1 เงินเดือน 3,457,920 บาท","01101-1 อัตราเดิม 8 อัตรา 3,054,320 บาท","3,054,320"
1,งาน: อํานวยการและบริหารสํานักงานเขต,"1. งบบุคลากร 4,028,000 บาท","1.1 เงินเดือน 3,457,920 บาท","01102-1 เงินเลื่อนขั้น 183,280 บาท","183,280"
2,งาน: อํานวยการและบริหารสํานักงานเขต,"1. งบบุคลากร 4,028,000 บาท","1.1 เงินเดือน 3,457,920 บาท","01106-1 เงินประจําตําแหน่ง 101,760 บาท","101,760"
3,งาน: อํานวยการและบริหารสํานักงานเขต,"1. งบบุคลากร 4,028,000 บาท","1.1 เงินเดือน 3,457,920 บาท","01107-1 เงินค่าตอบแทนรายเดือนของข้าราชการ 118,...","118,560"
4,งาน: อํานวยการและบริหารสํานักงานเขต,"1. งบบุคลากร 4,028,000 บาท","1.2 ค่าจ้างประจํา 507,200 บาท","01201-1 อัตราเดิม 2 อัตรา 472,040 บาท","472,040"
...,...,...,...,...,...
381,งาน: งบประมาณโรงเรียน,"5. งบรายจ่ายอื่น 5,015,200 บาท",07109-1 (4) ค่าใช้จ่ายในการพัฒนาคุณภาพการดําเน...,NaN,"29,800"
382,งาน: งบประมาณโรงเรียน,"5. งบรายจ่ายอื่น 5,015,200 บาท",07124-1 (5) ค่าใช้จ่ายในการจัดประชุมสัมมนาคณะก...,NaN,"81,300"
383,งาน: งบประมาณโรงเรียน,"5. งบรายจ่ายอื่น 5,015,200 บาท",07125-1 (6) ค่าใช้จ่ายในการสัมมนาประธานกรรมการ...,NaN,"21,300"
384,งาน: งบประมาณโรงเรียน,"5. งบรายจ่ายอื่น 5,015,200 บาท",07126-1 (7) ค่าใช้จ่ายในการส่งเสริมสนับสนุนให้...,NaN,"117,900"
